## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [52]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [53]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [54]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [55]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [56]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [57]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
# Answer below:
yelp['normal'] = yelp.text.map(lambda x: normalize(x))
yelp

,text,sentiment,normal
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust not good
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price
...,...,...,...
971,I think food should have flavor and texture an...,0,think food flavor textur lack
972,Appetite instantly gone.,0,appetit instantli gone
973,Overall I was not impressed and would not go b...,0,overal not impress would not go back
974,The whole experience was underwhelming and I t...,0,whole experi underwhelm think we ll go ninja s...


Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [59]:
# Answer below:
from tensorflow.keras.preprocessing.text import one_hot
yelp['encode'] = [one_hot(yelp, 1000) for yelp in yelp['normal']]
yelp

,text,sentiment,normal,encode
0,Wow... Loved this place.,1,wow love place,"[59, 325, 481]"
1,Crust is not good.,0,crust not good,"[265, 36, 144]"
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti,"[36, 268, 274, 448]"
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...,"[68, 419, 414, 78, 102, 250, 77, 318, 325, 213]"
4,The selection on the menu was great and so wer...,1,select menu great price,"[318, 198, 376, 277]"
...,...,...,...,...
971,I think food should have flavor and texture an...,0,think food flavor textur lack,"[382, 78, 64, 274, 455]"
972,Appetite instantly gone.,0,appetit instantli gone,"[497, 225, 405]"
973,Overall I was not impressed and would not go b...,0,overal not impress would not go back,"[295, 36, 234, 406, 36, 83, 426]"
974,The whole experience was underwhelming and I t...,0,whole experi underwhelm think we ll go ninja s...,"[130, 195, 285, 382, 163, 234, 83, 338, 271, 3..."


Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [60]:
# Answer below:
from tensorflow.keras.preprocessing.sequence import pad_sequences

ind_var = pad_sequences(yelp['encode'])
ind = pd.DataFrame(ind_var)
yelp = pd.concat([yelp,ind],axis=1)
yelp = yelp.drop('encode',axis=1)

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [83]:
# Answer below:
from sklearn.model_selection import train_test_split
y = yelp.sentiment
X = ind_var

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [84]:
# Answer below:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

max_words = np.max(ind_var)
model = Sequential()
model.add(Embedding(500, 100, input_length=ind_var.shape[1]))
model.add(SimpleRNN(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 83, 100)           50000     
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 32)                4256      
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 9         
Total params: 54,529
Trainable params: 54,529
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [85]:
# Answer below:

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=128)

Epoch 1/50
7/7 [==============================] - 1s 81ms/step - loss: 0.6919 - accuracy: 0.5390 - val_loss: 0.6965 - val_accuracy: 0.4337
Epoch 2/50
7/7 [==============================] - 0s 37ms/step - loss: 0.6666 - accuracy: 0.6373 - val_loss: 0.6978 - val_accuracy: 0.4490
Epoch 3/50
7/7 [==============================] - 0s 36ms/step - loss: 0.6341 - accuracy: 0.7135 - val_loss: 0.6901 - val_accuracy: 0.5357
Epoch 4/50
7/7 [==============================] - 0s 37ms/step - loss: 0.5764 - accuracy: 0.8275 - val_loss: 0.6898 - val_accuracy: 0.5306
Epoch 5/50
7/7 [==============================] - 0s 36ms/step - loss: 0.5103 - accuracy: 0.9016 - val_loss: 0.6821 - val_accuracy: 0.5714
Epoch 6/50
7/7 [==============================] - 0s 37ms/step - loss: 0.4449 - accuracy: 0.9235 - val_loss: 0.6830 - val_accuracy: 0.5867
Epoch 7/50
7/7 [==============================] - 0s 37ms/step - loss: 0.3796 - accuracy: 0.9236 - val_loss: 0.6944 - val_accuracy: 0.5816
Epoch 8/50
7/7 [===========